- 学修番号: 21445095
- 氏名: 小田龍司

# 課題 1.1　線形回帰分析

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

autos = pd.read_csv('data/imports-85.data', na_values='?')
autos.columns = ['symboling', 'normalized_losses', 'make', 'fuel_type', 'aspiration',
                    'num_of_doors', 'body_style', 'drive_wheels', 'engine_location',
                    'wheel_base', 'length','width', 'height', 'curb_weight',
                    'engine_type', 'num_of_cylinders', 'engine_size', 'fuel_system',
                    'bore', 'stroke', 'compression_ratio', 'horsepower', 'peak_rpm',
                    'city_mpg', 'highway_mpg','price']
# 表示する最大列数の設定
pd.set_option('display.max_columns', len(autos.columns))
# 欠損値 NaN がある行の除去
autos = autos.dropna()
# 確認
autos

,symboling,normalized_losses,make,fuel_type,aspiration,num_of_doors,body_style,drive_wheels,engine_location,wheel_base,length,width,height,curb_weight,engine_type,num_of_cylinders,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
2,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
3,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0
5,1,158.0,audi,gas,std,four,sedan,fwd,front,105.8,192.7,71.4,55.7,2844,ohc,five,136,mpfi,3.19,3.40,8.5,110.0,5500.0,19,25,17710.0
7,1,158.0,audi,gas,turbo,four,sedan,fwd,front,105.8,192.7,71.4,55.9,3086,ohc,five,131,mpfi,3.13,3.40,8.3,140.0,5500.0,17,20,23875.0
9,2,192.0,bmw,gas,std,two,sedan,rwd,front,101.2,176.8,64.8,54.3,2395,ohc,four,108,mpfi,3.50,2.80,8.8,101.0,5800.0,23,29,16430.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,-1,95.0,volvo,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,55.5,2952,ohc,four,141,mpfi,3.78,3.15,9.5,114.0,5400.0,23,28,16845.0
200,-1,95.0,volvo,gas,turbo,four,sedan,rwd,front,109.1,188.8,68.8,55.5,3049,ohc,four,141,mpfi,3.78,3.15,8.7,160.0,5300.0,19,25,19045.0
201,-1,95.0,volvo,gas,std,four,sedan,rwd,front,109.1,188.8,68.9,55.5,3012,ohcv,six,173,mpfi,3.58,2.87,8.8,134.0,5500.0,18,23,21485.0
202,-1,95.0,volvo,diesel,turbo,four,sedan,rwd,front,109.1,188.8,68.9,55.5,3217,ohc,six,145,idi,3.01,3.40,23.0,106.0,4800.0,26,27,22470.0


In [2]:
# price 以外の量的データのラベルを昇順にソート
qd_labels = autos.describe().columns.drop('price').sort_values()
# 確認
qd_labels

Index(['bore', 'city_mpg', 'compression_ratio', 'curb_weight', 'engine_size',
       'height', 'highway_mpg', 'horsepower', 'length', 'normalized_losses',
       'peak_rpm', 'stroke', 'symboling', 'wheel_base', 'width'],
      dtype='object')

<hr>

### 1.1.1 VIFが5以上の説明変数を除外

price 以外の量的データである説明変数について、昇順にソートしたラベル qd_labels の先頭からVIFが5以上の変数を除外し、残った説明変数について、説明変数のVIFを行とする pandas DataFrame を作成し、表示しなさい。
- 結果は、行ラベルは説明変数、列ラベルはVIFのn行1列の DataFrame

In [3]:
#決定係数R^2を計算
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# 目的変数 y_labelを引数で指定、それ以外を説明変数として決定係数を計算
def multi_regression_errors(df, y_label):
    X = df[qd_labels].drop([y_label], axis=1)
    Y = df[y_label]
    model = LinearRegression()
    model.fit(X, Y)
    Y_predict = model.predict(X)
    return r2_score(Y, Y_predict)

# qd_labels の各要素を目的変数としてVIFを計算
result_df = pd.DataFrame()
for y_label in qd_labels:
    r2 = multi_regression_errors(autos, y_label)
    vif = 1/(1-r2)
    
    # vifが5以上の場合代入しない
    if(vif >= 5): continue
    result_df.at[y_label, ["VIF"]] = 1/(1-r2)

result_df

,VIF
bore,2.343116
compression_ratio,2.579409
height,2.715781
normalized_losses,1.992013
peak_rpm,1.960272
stroke,1.484682
symboling,2.454976


<hr>

### 1.1.2 決定係数

すべての質的データの説明変数と1.1.1で残った説明変数を合わせて説明変数、price を目的変数として線形重回帰モデルを作成し、そのモデルの決定係数を求めなさい。<br>
小数点以下4桁目を四捨五入し、小数点以下3桁まで答えること（0.7625ならば0.763が答え）。

In [4]:
# 四捨五入用のライブラリ
from decimal import Decimal, ROUND_HALF_UP

# 不要な説明変数をドロップ
droped_index = qd_labels.drop(result_df.index)
df = autos.drop(droped_index, axis=1)

# 質的データのダミー変数化
df = pd.get_dummies(data=df, drop_first=True)

# 重回帰分析
X = df.drop(['price'], axis=1)
Y = df.price
model = LinearRegression()
model.fit(X, Y)
Y_predict = model.predict(X)

# 決定係数の計算
r2 = r2_score(Y, Y_predict)
r2 = Decimal(str(r2)).quantize(Decimal('0.001'), rounding=ROUND_HALF_UP)
print(r2)

0.958


<hr>

### 1.1.3 回帰係数

1.1.2で作成したモデルの回帰係数を、説明変数と回帰係数を行とする pandas DataFrame を作成し、降順で表示しなさい（行数が多い場合、途中の表示が省略されないようにすること）。

In [5]:
coef = pd.DataFrame(model.coef_.reshape(1, 46), columns=X.columns, index=['coef'])
coef.sort_values(by='coef',ascending=False, axis=1)

,make_jaguar,fuel_system_mfi,fuel_system_idi,fuel_system_mpfi,make_mercedes-benz,make_porsche,bore,fuel_system_2bbl,aspiration_turbo,fuel_system_spdi,drive_wheels_rwd,height,normalized_losses,...,engine_type_l,fuel_type_gas,make_nissan,make_mitsubishi,make_chevrolet,make_plymouth,make_dodge,make_toyota,body_style_hatchback,body_style_hardtop,body_style_wagon,body_style_sedan,num_of_cylinders_four
coef,8208.836794,6142.990397,4860.911914,4569.009609,4342.150877,3656.393469,3231.024586,3157.433043,2757.845059,2354.848724,1966.775353,14.864243,12.208257,...,-4840.907259,-4860.911914,-5089.498595,-5414.973266,-5551.664785,-5794.21287,-6020.5316,-6302.696691,-7044.581188,-7225.201313,-7472.06792,-7494.267165,-7740.655679
